##Welcome to LLM Finetuning Assignment
We start by installing some libraries.
We'll use unsloth, which is a library based on HuggingFace that is used to efficiently fine-tune LLM models in resource constrained environment.


In [ ]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

Found existing installation: unsloth 2024.9.post4
Uninstalling unsloth-2024.9.post4:
  Successfully uninstalled unsloth-2024.9.post4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-8tqjx1lu/unsloth_a300c10352f844eb9f6f57777e998ae6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-8tqjx1lu/unsloth_a300c10352f844eb9f6f57777e998ae6
  Resolved https://github.com/unslothai/unsloth.git to commit 79a2112ca4a775ce0b3cb75f5074136cb54ea6df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.9.post4-py3-none-any.whl size=165366 sha256=b2db78d2fd433f11482804ef768e99e464a08a270d6e0b3c952a2fb094ef07e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ddrrf8d5/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Ensure a distinct pad token is set
if tokenizer.pad_token is None or tokenizer.pad_token == tokenizer.eos_token:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a distinct pad token if needed

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2024-10-11 01:46:08,545] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# !zip -r model_files.zip /root/.cache/huggingface/hub


In [ ]:
# from google.colab import files
# files.download('model_files.zip')

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # Read the paper on low rank adaption to understand what layers to target https://arxiv.org/abs/2106.09685
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    random_state = 3407,
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Applying a chat template
Read and understand the chat templates of different LLMs, and add to your prompt and data the appropriate chat template of LLM.

In [ ]:
# from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "phi-3",
#     mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
# )

# def formatting_prompts_func(examples):
#     ## Read about chat templates, and determine a suitable chat template for your problem statement
#     convos = examples["conversations"]
#     texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
#     return { "text" : texts, }
# pass

# from datasets import load_dataset
# dataset = NotImplemented
# dataset = NotImplemented
# ## Your data at the end should have a "text" column that has your data

In [ ]:
import pandas as pd
from datasets import Dataset
import csv
from unsloth.chat_templates import get_chat_template

# Define your custom template
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant specializing in helping people find jobs.\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

# Load the CSV file
csv.field_size_limit(1000000)  # Adjust this value if needed
df = pd.read_csv('/content/drive/My Drive/conversational_job_data_train_v2.csv', encoding='iso-8859-1', quoting=csv.QUOTE_MINIMAL)

def create_conversations(row):
    return [
        {"from": "human", "value": row['Inputs'].strip()},
        {"from": "gpt", "value": row['Outputs'].strip()}
    ]

# Transform the data
conversations = df.apply(create_conversations, axis=1).tolist()

# Create a dataset
dataset = Dataset.from_dict({"conversations": conversations})

tokenizer = get_chat_template(
    tokenizer,
    chat_template = (unsloth_template, unsloth_eos_token,),
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"},
    map_eos_token = True,
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Apply the formatting function
formatted_dataset = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/3684 [00:00<?, ? examples/s]

In [ ]:
# Tokenize formatted dataset to prepare for model input
def tokenize_and_prepare_inputs(examples):
    # Tokenize the text while adding padding and returning the attention mask
    tokenized = tokenizer(
        examples["text"],
        padding=True,             # Pad to the longest sequence in the batch
        truncation=True,          # Truncate to the maximum sequence length
        return_tensors="pt",      # Return as PyTorch tensors
        return_attention_mask=True  # Generate attention masks for the tokens
    )
    return {
        "input_ids": tokenized["input_ids"].tolist(),  # Convert tensors to lists for compatibility with datasets
        "attention_mask": tokenized["attention_mask"].tolist()
    }

# Apply the tokenization function to the dataset
tokenized_dataset = formatted_dataset.map(
    tokenize_and_prepare_inputs,
    batched=True,
    remove_columns=formatted_dataset.column_names
)


Map:   0%|          | 0/3684 [00:00<?, ? examples/s]

## Sample of Chat template
The executed cell below is what a chat template looks like. Notice the <|end|>, <|user|>

In [ ]:
print(formatted_dataset[5]["text"])

<s>You are a helpful assistant specializing in helping people find jobs.
>>> User: Give me the information of available jobs in finland.
>>> Assistant: Job available in finland
Job-Title: senior frontend developereloper
Salary: Over $150K
Description: the role is senior frontend position at a company in helsinki finland working in the emobility sector the company is looking to take action on the climate crisis and reduce the co2 emissions by making electric vehicles more convenient<|endoftext|>



In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
!pip install deepspeed
from deepspeed.utils import logger
from deepspeed.utils.logging import should_log_le
from deepspeed.ops.op_builder import CPUAdamBuilder


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,  # Use the tokenized dataset
    dataset_text_field="input_ids",  # Use input_ids as the dataset text field for training
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        max_steps=60,
        learning_rate=5e-5,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
    ),
)
 # Read about SFTT trainer and put the arguments here https://huggingface.co/docs/trl/en/sft_trainer

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,684 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 60
 "-____-"     Number of trainable parameters = 29,884,416
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Below is a sample inference code. Make the necessary changes.


In [ ]:
from unsloth.chat_templates import get_chat_template
import torch

# Step 1: Update Tokenizer with Chat Template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="phi-3",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},  # ShareGPT style
)

# Step 2: Prepare the Model for Inference
FastLanguageModel.for_inference(model)

# Step 3: Create the Messages
messages = [
    {"from": "human", "value": "What jobs are open in New York?"},
]

# Step 4: Apply the Chat Template and Tokenize
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Must add for generation
    return_tensors="pt",  # Return as PyTorch tensor
    add_eos_token=True
)

# Debug step to check shapes
print(f"Type of inputs: {type(inputs)}")  # Should be <class 'torch.Tensor'>
print(f"Shape of inputs: {inputs.shape}")  # Should be [1, sequence_length]

# Move input tensors to the appropriate device (e.g., GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_ids = inputs.to(device)

# Step 5: Manually Create the Attention Mask
# An attention mask of 1s for all tokens (since we assume no padding here)
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)

# Step 6: Generate the Output with Attention Mask
outputs = model.generate(
    input_ids=input_ids,                # Use the inputs tensor directly
    attention_mask=attention_mask,      # Manually created attention mask
    max_new_tokens=256,
    use_cache=True
)

# Step 7: Decode the Outputs
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_outputs)


Type of inputs: <class 'torch.Tensor'>
Shape of inputs: torch.Size([1, 11])
["What jobs are open in New York? As of my knowledge cutoff in early 2023, I am unable to provide real-time job listings or access to current job market data. However, I can suggest ways to find out about open jobs in New York:\n\n1. **Job Boards**: Websites like Indeed, Glassdoor, and LinkedIn often have listings for jobs in New York. You can use their filters to specify the location and job type.\n\n2. **Company Websites**: Many companies have their own careers pages where they post job openings. You can visit the websites of companies you're interested in and check their job listings.\n\n3. **Local Job Centers**: The New York State Department of Labor has a website where you can find information about jobs in the state.\n\n4. **Social Media**: LinkedIn and Twitter can be great resources for finding job openings. You can follow companies and job-related hashtags in New York.\n\n5. **Recruitment Agencies**: So

## Read documentation of BERT Score and ROUGE and write evaluation metrics
https://huggingface.co/spaces/evaluate-metric/bertscore

https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
!pip install evaluate
!pip install bert_score
!pip install rouge_score

In [ ]:
from evaluate import load

# Load the BERTScore metric
bertscore = load("bertscore")

# Load your test dataset
csv.field_size_limit(1000000)  # Adjust this value if needed
df = pd.read_csv('/content/drive/My Drive/conversational_job_data_test_v2.csv', encoding='iso-8859-1', quoting=csv.QUOTE_MINIMAL)

# Extract the relevant parts from Inputs and Outputs
def extract_query(text):
    return text.split('Inputs:')[-1].strip()

def extract_job_info(text):
    return text.split('Outputs:')[-1].strip()

# Prepare predictions and references
predictions = df['Outputs'].apply(extract_job_info).tolist()
references = df['Inputs'].apply(extract_query).tolist()

# Compute BERTScore
results = bertscore.compute(
    predictions=predictions,
    references=references,
    model_type="distilbert-base-uncased",
    lang="en"
)

# Print the results
print(results)

# Calculate and print average scores
avg_precision = sum(results['precision']) / len(results['precision'])
avg_recall = sum(results['recall']) / len(results['recall'])
avg_f1 = sum(results['f1']) / len(results['f1'])

print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1: {avg_f1}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'precision': [0.709296703338623, 0.7087810039520264, 0.7040260434150696, 0.6979368925094604, 0.696902334690094, 0.7100161910057068, 0.7423431873321533, 0.7272070050239563, 0.7396384477615356, 0.8876912593841553, 0.8830538988113403, 0.8869010210037231, 0.7091977596282959, 0.7049278020858765, 0.7050191164016724, 0.6744893789291382, 0.6733591556549072, 0.6879011392593384, 0.6822935342788696, 0.6670603156089783, 0.6714376211166382, 0.8727484941482544, 0.86418616771698, 0.8730281591415405, 0.7286820411682129, 0.7275493741035461, 0.7252059578895569, 0.6811131834983826, 0.6798338294029236, 0.6915456652641296, 0.6701032519340515, 0.6549965143203735, 0.6634742617607117, 0.8884329795837402, 0.8817054033279419, 0.8882402777671814, 0.7142195701599121, 0.7142018675804138, 0.7080184817314148, 0.6979368925094604, 0.696902334690094, 0.7100161910057068, 0.7423431873321533, 0.7272070050239563, 0.7396384477615356, 0.8876912593841553, 0.8830538988113403, 0.8869010210037231, 0.6720988750457764, 0.67204886

In [ ]:
# Load the ROUGE metric
rouge = load('rouge')

# Load your test dataset
csv.field_size_limit(1000000)  # Adjust this value if needed
df = pd.read_csv('conversational_job_data_test_v2.csv', encoding='iso-8859-1',quoting=csv.QUOTE_MINIMAL)

# Extract the relevant parts from Inputs and Outputs
def extract_query(text):
    return text.split('Inputs:')[-1].strip()

def extract_job_info(text):
    return text.split('Outputs:')[-1].strip()

# Prepare predictions and references
predictions = df['Outputs'].apply(extract_job_info).tolist()
references = df['Inputs'].apply(extract_query).tolist()

# Compute ROUGE without aggregation
results_no_agg = rouge.compute(predictions=predictions, references=references, use_aggregator=False)

# Compute ROUGE with aggregation
results_agg = rouge.compute(predictions=predictions, references=references, use_aggregator=True)

# Print results without aggregation (first few entries)
print("ROUGE scores without aggregation (first 5 entries):")
for key in results_no_agg:
    print(f"{key}: {results_no_agg[key][:5]}")

# Print results with aggregation
print("\nROUGE scores with aggregation:")
for key, value in results_agg.items():
    print(f"{key}: {value}")

# Calculate and print average scores for non-aggregated results
print("\nAverage ROUGE scores (from non-aggregated results):")
for key in results_no_agg:
    avg_score = sum(results_no_agg[key]) / len(results_no_agg[key])
    print(f"Average {key}: {avg_score}")

ROUGE scores without aggregation (first 5 entries):
rouge1: [0.1702127659574468, 0.15053763440860216, 0.1702127659574468, 0.09375, 0.125]
rouge2: [0.0, 0.0, 0.0, 0.06451612903225806, 0.09677419354838708]
rougeL: [0.10638297872340427, 0.08602150537634408, 0.10638297872340427, 0.09375, 0.125]
rougeLsum: [0.14893617021276595, 0.12903225806451615, 0.14893617021276595, 0.09375, 0.125]

ROUGE scores with aggregation:
rouge1: 0.29491806128318454
rouge2: 0.24701729452362509
rougeL: 0.2813969337780525
rougeLsum: 0.2894696024275836

Average ROUGE scores (from non-aggregated results):
Average rouge1: 0.2948228834295634
Average rouge2: 0.24665427245330873
Average rougeL: 0.28150636730214207
Average rougeLsum: 0.2897388027349812
